## Create k-mer matrix using a coordinadte sparse matrix
This was the only way I could work how to do this without defining the size of the matric before I knew how many unique kmers there are.  
The text files from dsk are in data_dir.

Presence/absence matrix. ones being stored as int64 - should probably change this to compress the data. - now int8 

In [5]:
import numpy as np
from pathlib import Path
import pandas as pd
import glob
import os
import pickle
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
import gzip

### Load k-mers from DSK output

Point this to the folder containing all the .txt files

In [6]:
data_dir = 'kmers/'
filenames = sorted(glob.glob(os.path.join(data_dir, '*.txt')))
print (len(filenames), filenames[0])

1494 kmers/GCF_000195955.2_ASM19595v2_genomic.txt


In [3]:
filenames[0]

'kmers/GCF_000195955.2_ASM19595v2_genomic.txt'

In [ ]:
boolarr = np.array([[0, 0, 1], [1, 0, 1], [1, 0, 1]], dtype=np.bool)
np.sum(boolarr, axis=0)
datas = np.ones(10, dtype=np.bool)
datas

In [7]:
def extract_X (filenames):

    strain_index = {} # {strain:i}
    kmer_index = {}   # {kmer:j}
    kmer_count =0     # keeps track of total number of kmers 

    row = []
    col = []
    data = []
    for i,filename in enumerate(filenames):
        if i % 50 == 0:
            print(f'{i+1} of {len(filenames)}  {filename}')
        df = pd.read_csv (filename, header=None, sep=' ')
        strain = Path(filename).stem.split('_')[1]
        strain_index[strain] = i
        kmers = list(df[df.columns[0]])
        for kmer in kmers:
            try:
                j  = kmer_index[kmer]
            except KeyError:
                j = kmer_count
                kmer_count += 1
                kmer_index [kmer] = j
            row.append(i)
            col.append(j)
            #data.append(1)


    rows  = np.array(row)
    cols  = np.array(col)
    datas = np.ones(len(rows), dtype=np.bool) #np.dtype('>i1')
    
    M = len(strain_index)
    N = len(kmer_index)
    print('data extracted', M,N)
    X = coo_matrix((datas,(rows,cols)), shape=(M,N))
    return  strain_index, kmer_index, X

In [ ]:
%%time
strains,kmer_index, S_coo = extract_X (filenames)
S_coo

1 of 1494  kmers/GCF_000195955.2_ASM19595v2_genomic.txt
51 of 1494  kmers/GCF_000649875.1_Myco_tube_MD19964_V1_genomic.txt
151 of 1494  kmers/GCF_000652215.1_Myco_tube_TKK_03_0118_V1_genomic.txt
201 of 1494  kmers/GCF_000653375.1_Myco_tube_TKK_05SA_0010_V1_genomic.txt
251 of 1494  kmers/GCF_000654515.1_Myco_tube_M1021_V1_genomic.txt
301 of 1494  kmers/GCF_000655535.1_Myco_tube_XTB13-208_V1_genomic.txt
351 of 1494  kmers/GCF_000656575.1_Myco_tube_UT0098_V1_genomic.txt
401 of 1494  kmers/GCF_000657615.1_Myco_tube_NRITLD33_V1_genomic.txt
451 of 1494  kmers/GCF_000658925.1_Myco_tube_BTB10-258_V1_genomic.txt


### Convert to csr for machine learning

In [ ]:
X = csr_matrix(S_coo)
X.data.nbytes

In [4]:
!ls


dsk_test.ipynb			   svm_DNA31.ipynb
Example_extract_kmers.ipynb	   TB_extract2sparse.ipynb
example.txt.bz2			   tbmetafiles
extract_matrix.ipynb		   tbmetafiles.txt
features.pkl			   test_kmer.ipynb
finding_kmers.ipynb		   test_stringkenal.ipynb
frans_extract_sparse_matrix.ipynb  txt_archive
frans_svm.ipynb			   X_all_20_1470.bz2
gpk.py				   X_all_50_1450.bz2
kmers				   X_meta_all_20_1470.bz2
labels.pkl			   X_meta_all_50_1450.bz2
PATRIC_genome_sequence.fasta	   X_meta_pyrazinamide_5_140.bz2
ReadX.ipynb			   X_pyrazinamide_5_140.bz2
stringkernel.py			   X_rifampin_20_1470.bz2


In [10]:
np.shape(X)

(1494, 7896888)

## First filter 
Just to speed up next steps

In [35]:
minimum = 20
maximum = 1470

Z =np.sum(X, axis=0)
keep = np.where((np.logical_and(Z>=minimum, Z<=maximum)))
ind =(keep[1].tolist())
X_1 = X[:,ind]

#  Update kmer_index to match filtered X

index2kmers = {v:k for k,v in kmer_index.items()}
filtered_1_kmers = {index2kmers[index]:i for  i,index in enumerate (ind)}

print(X_1.shape,len(filtered_1_kmers))


(1494, 690051) 690051


In [40]:
np.max(Z[keep])

1470

In [ ]:
import bz2
import os

# Output X and y  
ab = 'all'
fileout1 = f'X_{ab}_{minimum}_{maximum}.bz2'
fileout2 = f'X_meta_{ab}_{minimum}_{maximum}.bz2'
output1 = bz2.BZ2File(fileout1, 'wb', compresslevel=5)
try:
    pickle.dump(X_1,output1)
finally:
    output1.close()

print(os.system(fileout1))


data = ({'antibiotic':ab,'strain index': strains,'kmer_index':filtered_1_kmers})
output2 = bz2.BZ2File(fileout2, 'wb', compresslevel=5)
try:
    pickle.dump(data,output2)
finally:
    output2.close()

os.system(fileout2)

32256


##  Filter on antibiotic datasets 
In TBmetadata

In [11]:
#amr/TBmetadata/isoniazid.csv
antibiotics = ['pyrazinamide','isoniazid','rifampin','ethambutol']
for ab in antibiotics:
    
#ab = antibiotics[3]
    mdata = pd.read_csv(f'../TBmetadata/{ab}.csv')
    print(ab, len(mdata))
#     TB = (list(mdata[mdata.columns[0]]))
#     y = list(mdata[mdata.columns[2]])
#     TB = ['000'+str(b) for b in TB]


pyrazinamide 144
isoniazid 1427
rifampin 1461
ethambutol 1198


In [ ]:
#  Filter for stains in AMR dataset 
keep_strains = [i for strain,i in strains.items() if strain in TB]
X_filt = X[keep_strains, :]

index2strains = {v:k for k,v in strains.items()}
filtered_strains = {index2strains[index]:i for  i,index in enumerate (keep_strains)}


### Filter kmers on min and max number of genomes they occur in.
Because this is presense/absense then the sum of each column is the number of genomes each kmer is present in.


In [ ]:
minimum = 20  # should change this for small dataset - pyrazinamide
maximum = 1470

Z =np.sum(X_filt, axis=0)
keep = np.where((np.logical_and(Z>=minimum, Z<=maximum)))
ind =(keep[1].tolist())
X_filt_2way = X_filt[:,ind]

#  Update kmer_index to match filtered X

index2kmers = {v:k for k,v in kmer_index.items()}
filtered_kmers = {index2kmers[index]:i for  i,index in enumerate (ind)}

fileout = f'X_{ab}_{minimum}_{maximum}.bz2'
print(f'Filename: {fileout}, antibiotic {ab}, X.shape  {X_filt_2way.shape},No of kmers {len(filtered_kmers)}')




In [ ]:
for ab in antibiotics:
    
    mdata = pd.read_csv(f'../TBmetadata/{ab}.csv')
    print(ab, len(mdata))

In [14]:
#amr/TBmetadata/isoniazid.csv
antibiotics = ['pyrazinamide','isoniazid','rifampin','ethambutol']
ab = antibiotics[0]

#for ab in antibiotics[1:]:
mdata = pd.read_csv(f'../TBmetadata/{ab}.csv')
print(ab, len(mdata))
TB = (list(mdata[mdata.columns[0]]))
#y = list(mdata[mdata.columns[2]])
TB = ['000'+str(b) for b in TB]

#  filter on strains in dataset
keep_strains = [i for strain,i in strains.items() if strain in TB]
X_filt = X[keep_strains, :]
index2strains = {v:k for k,v in strains.items()}
filtered_strains = {index2strains[index]:i for  i,index in enumerate (keep_strains)}

print (f' Number of strains: {len(filtered_strains)}')

# filter kmers on number of genomes it appears in
minimum = 20  # should change this for small dataset - pyrazinamide
maximum = 1470

if len(TB) < 1000:
    minimum = 5
    maximum = 140

Z =np.sum(X_filt, axis=0)
keep = np.where((np.logical_and(Z>=minimum, Z<=maximum)))
ind =(keep[1].tolist())
X_filt_2way = X_filt[:,ind]

#  Update kmer_index to match filtered X
index2kmers = {v:k for k,v in kmer_index.items()}
filtered_kmers = {index2kmers[index]:i for  i,index in enumerate (ind)}
print (f' Number of kmers {len(filtered_kmers)}')


pyrazinamide 144
 Number of strains: 144
 Number of kmers 350739


In [28]:
type(X_filt_2way)

scipy.sparse.csr.csr_matrix

In [24]:
strain_list = [ index2strains[i] for i in range(len(filtered_strains))]
kmer_list = [index2kmers[i] for i in range(len(filtered_kmers))]

In [29]:
df = pd.DataFrame(data =X_filt_2way.todense(), index = strain_list, columns = kmer_list)

In [30]:
df.head()

,AAAAAACCCTGCACTTCATCGACAATGCGAT,AAAAAATAAGGCGGCAAAAATATCCCGTCAG,AAAAAAGTCGTCCCAGTGGATGAGAATCACG,AAAAACCCTCGGCCGCAAAAGGCTTGCGAAT,AAAAACCCTGCACTTCATCGACAATGCGATC,AAAAACGAACAACGAGCCCGCCGCGCATTGA,AAAAACGAAGACGTCATCGTGCGAGAGCTCA,AAAAATAAGGCGGCAAAAATATCCCGTCAGA,AAAAATATCCCGTCAGACGGTCACGTCATAC,AAAAATCACGGAAGGCGACTCCACTTTCGAT,...,CGCGCAACACAATCCGATGCCCGTCGAGATC,CGCGCAACGACGTTGTTCCTTAGAGCGTGAC,CGCGCAACGAGCACAAGCGGCACCGCTATGC,CGCGCAATATCGCTATCGGGTTCGTCGGCGC,CGCGCAAGGTCCGGCTGGGCCCATCAGCGAA,CGCGCACACATCCGCTCGGTCACCGCGCGCC,CGCGCACACCTTCCGGTAGATGTCCAGTGTC,CGCGCACACCTTCTGGGACGACATCAAAGCC,CGCGCACCAACGCCTGCTGCGCGACGGCCGC,CGCGCACCACCGCGGTGATCAGGAAATGCGC
000195955.2,True,True,True,True,False,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
000240245.1,True,True,True,False,True,True,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False
000240265.1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
000240285.1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
000291765.1,True,True,True,False,True,True,True,False,True,True,...,False,False,False,False,False,False,False,False,False,False


In [33]:
   
# Output X and y  
fileout1 = f'X_df_{ab}_{minimum}_{maximum}.bbz2'
print(f'Filename: {fileout1} contains  X for antibiotic {ab}, X.shape  {X_filt_2way.shape},No of kmers {len(filtered_kmers)}')

# fileout2 = f'X_meta_{ab}_{minimum}_{maximum}.bz2'
# print(f'Filename: {fileout2} strain and kmer index and y, Number of strains {len(strains)},No of kmers {len(filtered_kmers)}')


Filename: X_df_pyrazinamide_5_140.bbz2 contains  X for antibiotic pyrazinamide, X.shape  (144, 350739),No of kmers 350739


In [35]:
# import joblib
# import os

# # Output X and y  
# filename = fileout
# #with open(filename, 'wb') as fo:  
# try:
#     joblib.dump(to_persist, filename + '.compressed', compress=True) 
# finally:
#     output.close()

# os.system(fileout1)

import bz2
import os
data = {'df':df} #({'antibiotic':ab,'strain index': strains,'kmer_index':kmer_index, 'y':y})
output = bz2.BZ2File(fileout1, 'wb', compresslevel=5)
try:
    pickle.dump(data,output)
finally:
    output.close()

os.system(fileout2)

NameError: name 'fileout2' is not defined

In [36]:
!ls -lh

total 2.6G
-rw-r--r-- 1 youn01f users  35K Oct  4 14:12 dsk_test.ipynb
-rw-r--r-- 1 youn01f users 9.5K Jul  3 12:39 Example_extract_kmers.ipynb
-rw-r--r-- 1 youn01f users   14 Oct  9 09:46 example.txt.bz2
-rw-r--r-- 1 youn01f users 141K Oct  6 12:02 extract_matrix.ipynb
-rw-r--r-- 1 youn01f users 817M Aug 22 10:49 features.pkl
-rw-r--r-- 1 youn01f users  35K Sep  4 12:31 finding_kmers.ipynb
-rw-r--r-- 1 youn01f users  31K Oct 25 13:01 frans_extract_sparse_matrix.ipynb
-rw-r--r-- 1 youn01f users  27K Sep  3 22:49 frans_svm.ipynb
-rw-r--r-- 1 youn01f users  597 Jul  3 12:39 gpk.py
drwxr-xr-x 3 youn01f users 124K Oct  4 18:54 kmers
-rw-r--r-- 1 youn01f users 4.6K Aug 22 10:49 labels.pkl
-rw-r--r-- 1 youn01f users 9.6M Jul  3 12:39 PATRIC_genome_sequence.fasta
-rw-r--r-- 1 youn01f users  12K Oct 18 12:04 ReadX.ipynb
-rw-r--r-- 1 youn01f users 5.2K Jul  3 12:39 stringkernel.py
-rw-r--r-- 1 youn01f users  91K Sep 20 13:48 svm_DNA31.ipynb
-rw-r--r-- 1 youn01f users  16K Oct  2 19:18 TB_extrac